In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import tensorflow as tf
import tensorflow.kerasras as keras

# data &feat

In [147]:
vocab_size = 20000 
maxlen = 200  

In [148]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

/Users/zhangjiaqi/workingspace/anaconda3/envs/ml3/lib/python3.7/site-packages/tensorflow_core/python/keras/datasets/imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/zhangjiaqi/workingspace/anaconda3/envs/ml3/lib/python3.7/site-packages/tensorflow_core/python/keras/datasets/imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


25000 Training sequences
25000 Validation sequences


# model

In [149]:
##transformer= 
## xa = layerNorm(multi_head_attention(x)+x)
## xb = layerNorm(MLP(xa)+xa)
## embed = token_embed + poistion_embed

In [150]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Embedding

In [151]:
tf.transpose?

In [152]:
tf.reshape?

In [153]:
tf.nn.softmax?

In [154]:
tf.shape?

In [155]:
class MultiHeadAttention_keras(keras.layers.Layer):
    '''
    process inputs by multi_head_attention
    inputs:[batch_size,seq_len,embed_dim]
    embed_dim = num_heads*embed_dim2
    self_attention[batch_size,num_heads,seq_len,seq_len]
    query,key,value:[batch_size,num_head,seq_len,embed_dim2]
    '''
    def __init__(self,embed_dim,num_heads = 10):
        super(MultiHeadAttention_keras,self).__init__()
        if(embed_dim%num_heads !=0):
            print('un match dim')
            return
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        ##query/key/value trans
        self.project_dim = self.embed_dim//num_heads
        ##trans [batch_size,seq_len,embed_dim] to [batch_size,seq_len,project_dim*num_heads]
        self.query = keras.layers.Dense(units=self.project_dim*self.num_heads,name='query_mlp')
        self.key = keras.layers.Dense(units=self.project_dim*self.num_heads,name='key_mlp')
        self.value = keras.layers.Dense(units=self.project_dim*self.num_heads,name='value_mlp')
        ##concat and trans
        self.concat = keras.layers.Dense(units=self.project_dim*self.num_heads,name='concat_head')
    
    def call(self,inputs):
        ##inputs:[batch_size,seq_len,embed_dim]
        query = self.key(inputs)
        key = self.key(inputs)
        value = self.value(inputs)
        
        ##query,key,value:[batch_size,seq_len,project_dim*num_heads]
        query = self.split_head(query)
        key = self.split_head(key)
        value = self.split_head(value)
        
        ## compute self-attention
        att_out = self.self_attention(query,key,value)#[batch_size,num_heads,seq_len,embed_dim2]
        att_out = tf.transpose(att_out,perm=[0,2,1,3])
        
        new_shape = tf.shape(inputs)
        print(new_shape)
        att_out = tf.reshape(att_out,shape=new_shape)
        out = self.concat(att_out)
        return out
        
    def split_head(self,x):
        ##x:[batch_size,seq_len,num_heads*embed_dim2]
        #new_shape = tf.shape(x)[:-1]#+tf.cast([self.num_heads,self.project_dim],tf.int32)
        new_shape=tf.cast([tf.shape(x)[0],tf.shape(x)[1],self.num_heads,self.project_dim],tf.int32)
        #print(new_shape)
        x = tf.reshape(x,new_shape) ##[batch_size,seq_len,num_heads,embed_dim2]
        x = tf.transpose(x,perm=[0,2,1,3])
        return x
    
    def self_attention(self,query,key,value):
        ##x:[batch_size,num_heads,seq_len,embed_dim2]
        attention_weight = tf.matmul(query,key,transpose_b=True)
        ##attentin_weight:[batch_size,num_heads,seq_len,seq_len]
        ##scaled-dot-product
        #print(attention_weight)
        scaled_ratio = tf.sqrt(tf.cast(tf.shape(query)[-1],tf.float32))
        attention_weight /= scaled_ratio
        ##attention_weight:[batch_size,num_heads,seq_len,seq_len]
        attention_weight = tf.nn.softmax(attention_weight,axis=-1)
        
        ##get attention_value
        att_out = tf.matmul(attention_weight,value)
        return att_out

In [156]:
query = np.random.random_sample((100,20,40)).astype(np.float32)

In [157]:
query.dtype

dtype('float32')

In [158]:
att_query = MultiHeadAttention_keras(40,num_heads=10)(query)

Tensor("multi_head_attention_keras_1/Shape_7:0", shape=(3,), dtype=int32)


In [159]:
##transformer block:
## xa = layerNorm(multi_head_attention(x)+x)
## xb = layerNorm(MLP(xa)+xa)

In [160]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self,embed_dim,num_heads,mlp_num,keep_prob = 0.1):
        super(TransformerBlock,self).__init__()
        self.att = MultiHeadAttention_keras(embed_dim,num_heads)
        self.ln1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.mlp = keras.models.Sequential([
            keras.layers.Dense(units=mlp_num,activation='relu'),
            keras.layers.Dense(units=embed_dim)
        ])
        
        self.dropout1 = keras.layers.Dropout(keep_prob)
        self.dropout2 = keras.layers.Dropout(keep_prob)
    
    def call(self,inputs,training=True):
        ##[batch_size,seq_len,embed_dim]
        att_out = self.att(inputs)
        att_out = self.dropout1(att_out,training=training)
        out1 = self.ln1(inputs+att_out)
        mlp_out = self.mlp(out1)
        mlp_out = self.dropout2(mlp_out,training=training)
        out2 = self.ln2(mlp_out+out1)
        return out2

In [161]:
query = np.random.random_sample((100,20,40)).astype(np.float32)
trans_query = TransformerBlock(embed_dim=40,num_heads=10,mlp_num=20)(query,True)

Tensor("transformer_block_1/multi_head_attention_keras_2/Shape_7:0", shape=(3,), dtype=int32)


In [162]:
##token_embed and position_embed

In [163]:
class TokenAndPositionEmbed(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbed, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

## model part

In [164]:
##transformer= 
## xa = layerNorm(multi_head_attention(x)+x)
## xb = layerNorm(MLP(xa)+xa)
## embed = token_embed + poistion_embed

In [165]:
embed_dim = 32 
num_heads = 2  
mlp_dim = 32 

In [166]:
#keras.backend.clear_session()

In [167]:
inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbed(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, mlp_dim)
x = transformer_block(x)
x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(20, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(2, activation="softmax")(x)


Tensor("transformer_block_2/multi_head_attention_keras_3/Shape_7:0", shape=(3,), dtype=int32)


In [168]:
nn = keras.Model(inputs=inputs, outputs=outputs)

In [169]:
nn.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
token_and_position_embed_1 ( (None, 200, 32)           646400    
_________________________________________________________________
transformer_block_2 (Transfo (None, 200, 32)           5408      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_9 (Dropout)          (None, 20)                0   

In [173]:
nn.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = nn.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 386s 15ms/sample - loss: 0.3868 - acc: 0.8117 - val_loss: 0.3023 - val_acc: 0.8718
Epoch 2/2
25000/25000 [==============================] - 442s 18ms/sample - loss: 0.2036 - acc: 0.9222 - val_loss: 0.3201 - val_acc: 0.8692


In [171]:
for layer in nn.layers:
    print(layer.name)

input_2
token_and_position_embed_1
transformer_block_2
global_average_pooling1d_1
dropout_8
dense_8
dropout_9
dense_9
